In [1]:
import sys
import yaml
import torch
import logging
from pathlib import Path

# Add the path to the directory containing the omnicell package
# Assuming the omnicell package is in the parent directory of your notebook
sys.path.append('..')  # Adjust this path as needed

import yaml
import torch
import logging
from pathlib import Path
from omnicell.config.config import Config, ETLConfig, ModelConfig, DatasplitConfig, EvalConfig, EmbeddingConfig
from omnicell.data.loader import DataLoader
from omnicell.constants import PERT_KEY, GENE_EMBEDDING_KEY, CONTROL_PERT
from train import get_model

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Configure paths
MODEL_CONFIG = ModelConfig.from_yaml("/orcd/data/omarabu/001/njwfish/omnicell/configs/models/linear_mean_model.yaml")
ETL_CONFIG = ETLConfig(name = "no_preprocessing", log1p = False, drop_unmatched_perts = True)
EMBEDDING_CONFIG = EmbeddingConfig(pert_embedding='GenePT')

SPLIT_CONFIG = DatasplitConfig.from_yaml("/orcd/data/omarabu/001/njwfish/omnicell/configs/splits/repogle_k562_essential_raw/random_splits/rs_accP_k562_ood_ss:ns_20_2_most_pert_0.1/split_0/split_config.yaml")
EVAL_CONFIG = EvalConfig.from_yaml("/orcd/data/omarabu/001/njwfish/omnicell/configs/splits/repogle_k562_essential_raw/random_splits/rs_accP_k562_ood_ss:ns_20_2_most_pert_0.1/split_0/eval_config.yaml")  # Set this if you want to run evaluations

# Load configuration from paths 
config = Config(model_config=ModelConfig,
                 etl_config=ETL_CONFIG, 
                 datasplit_config=SPLIT_CONFIG, 
                 eval_config=EVAL_CONFIG)

#Alternatively you can initialize the config objects manually as follows:
# etl_config = ETLConfig(name = XXX, log1p = False, drop_unmatched_perts = False, ...)
# model_config = ...
# embedding_config = ...
# datasplit_config = ...
# eval_config = ...
# config = Config(etl_config, model_config, datasplit_config, eval_config)


config.etl_config.pert_embedding = 'bioBERT'
config.etl_config.drop_unmatched_perts = True
# Set up device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Initialize data loader and load training data
loader = DataLoader(config)
adata, pert_rep_map = loader.get_training_data()

# Get dimensions and perturbation IDs
input_dim = adata.obsm['embedding'].shape[1]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pert_ids = adata.obs[PERT_KEY].unique()
gene_emb_dim = adata.varm[GENE_EMBEDDING_KEY].shape[1] if GENE_EMBEDDING_KEY in adata.varm else None

print(f"Data loaded:")
print(f"- Number of cells: {adata.shape[0]}")
print(f"- Input dimension: {input_dim}")
print(f"- Number of perturbations: {len(pert_ids)}")


2025-01-27 17:05:32,277 - INFO - Loading data catalogue from /orcd/data/omarabu/001/opitcho/omnicell/configs/catalogue/adamson_INCOMPLETE.yaml
2025-01-27 17:05:32,278 - INFO - Loading data catalogue from /orcd/data/omarabu/001/opitcho/omnicell/configs/catalogue/satija_IFNG_raw_INCOMPLETE.yaml
2025-01-27 17:05:32,280 - INFO - Loading data catalogue from /orcd/data/omarabu/001/opitcho/omnicell/configs/catalogue/satija_IFNB_raw.yaml
2025-01-27 17:05:32,281 - INFO - Loading data catalogue from /orcd/data/omarabu/001/opitcho/omnicell/configs/catalogue/satija_IFNB_HVG.yaml
2025-01-27 17:05:32,282 - INFO - Loading data catalogue from /orcd/data/omarabu/001/opitcho/omnicell/configs/catalogue/satija_IFNG_raw.yaml
2025-01-27 17:05:32,284 - INFO - Loading data catalogue from /orcd/data/omarabu/001/opitcho/omnicell/configs/catalogue/repogle_k562_essential_raw.yaml
2025-01-27 17:05:32,285 - INFO - Loading data catalogue from /orcd/data/omarabu/001/opitcho/omnicell/configs/catalogue/essential_gene_k

Using device: cuda


2025-01-27 17:05:35,587 - INFO - Loaded unpreprocessed data, # of data points: 310385, # of genes: 8563.
2025-01-27 17:05:35,588 - INFO - Preprocessing training data
2025-01-27 17:05:35,590 - INFO - Using identity features for perturbations
2025-01-27 17:05:35,700 - INFO - Removing observations with perturbations not in the dataset as a column
2025-01-27 17:05:35,890 - INFO - Removed 189 perturbations that were not in the dataset columns and 0 perturbations that did not have an embedding for a total of 189 perturbations removed out of an initial 2058 perturbations
/orcd/data/omarabu/001/opitcho/omnicell/notebooks/../omnicell/data/loader.py:178: ImplicitModificationWarning: Setting element `.obsm['embedding']` of view, initializing view as actual.
  adata.obsm["embedding"] = adata.X.toarray().astype('float32')
2025-01-27 17:06:11,639 - INFO - Doing OOD split


Data loaded:
- Number of cells: 279630
- Input dimension: 8563
- Number of perturbations: 1850


In [6]:

model = get_model(config.model_config.name, config.model_config, loader, pert_rep_map, input_dim, device, pert_ids, gene_emb_dim)

AttributeError: type object 'ModelConfig' has no attribute 'name'

In [5]:
model.train(adata)

NameError: name 'model' is not defined

In [4]:
import numpy as np

logger.info("Running evaluation")

# evaluate each pair of cells and perts
eval_dict = {}
for cell_id, pert_id, ctrl_data, gt_data in loader.get_eval_data():
    logger.debug(f"Making predictions for cell: {cell_id}, pert: {pert_id}")

    preds = model.make_predict(ctrl_data, pert_id, cell_id)
    eval_dict[(cell_id, pert_id)] = (ctrl_data.X.toarray(), gt_data.X.toarray(), preds)
    
if not config.etl_config['log1p']:
    for (cell, pert) in eval_dict:  
        ctrl_data, gt_data, pred_pert = eval_dict[(cell, pert)]
        eval_dict[(cell, pert)] =  (np.log1p(ctrl_data), np.log1p(gt_data), np.log1p(pred_pert))


2025-01-16 21:26:16,081 - INFO - Running evaluation
2025-01-16 21:26:24,940 - WARNING - No data found for cell: k562, pert: U2AF1 in repogle_k562_essential_raw, will skip evaluation


In [5]:
import scanpy as sc
from omnicell.evaluation.utils import get_DEGs, get_eval, get_DEG_Coverage_Recall, get_DEGs_overlaps
pval_threshold = 0.05
log_fold_change_threshold = 0.0

results_dict = {}
for (cell, pert) in eval_dict:  
    ctrl_data, gt_data, pred_pert = eval_dict[(cell, pert)]

    pred_pert = sc.AnnData(X=pred_pert)
    true_pert = sc.AnnData(X=gt_data)
    control = sc.AnnData(X=ctrl_data)

    logger.debug(f"Getting ground Truth DEGs for {pert} and {cell}")
    true_DEGs_df = get_DEGs(control, true_pert)
    signif_true_DEG = true_DEGs_df[true_DEGs_df['pvals_adj'] < pval_threshold]

    logger.debug(f"Number of significant DEGS from ground truth: {signif_true_DEG.shape[0]}")

    logger.debug(f"Getting predicted DEGs for {pert} and {cell}")
    pred_DEGs_df = get_DEGs(control, pred_pert)


    logger.debug(f"Getting evaluation metrics for {pert} and {cell}")
    r2_and_mse = get_eval(control, true_pert, pred_pert, true_DEGs_df, [100,50,20], pval_threshold, log_fold_change_threshold)

    logger.debug(f"Getting DEG overlaps for {pert} and {cell}")
    DEGs_overlaps = get_DEGs_overlaps(true_DEGs_df, pred_DEGs_df, [100,50,20], pval_threshold, log_fold_change_threshold)

    results_dict[(cell, pert)] = (r2_and_mse, DEGs_overlaps)



/home/njwfish/miniconda3/envs/omnicell/lib/python3.9/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/njwfish/miniconda3/envs/omnicell/lib/python3.9/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/njwfish/miniconda3/envs/omnicell/lib/python3.9/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/njwfish/miniconda3/envs/omnicell/lib/python3.9/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/njwfish/miniconda3/envs/omnicell/lib/python3.9/site-packag

In [8]:
for (cell, pert) in results_dict:
    r2_and_mse, DEGs_overlaps = results_dict[(cell, pert)]
    print(f"Cell: {cell}, Pert: {pert}")
    # print(f"R2 and MSE: {r2_and_mse}")
    print(f"DEGs Overlaps: {DEGs_overlaps}")
    print("-"*100)


Cell: k562, Pert: RPL15
DEGs Overlaps: {'Overlap_in_top_2159_DEGs': 935, 'Overlap_in_top_100_DEGs': 33, 'Overlap_in_top_50_DEGs': 11, 'Overlap_in_top_20_DEGs': 2, 'Jaccard': 0.35336356764928195}
----------------------------------------------------------------------------------------------------
Cell: k562, Pert: RPL4
DEGs Overlaps: {'Overlap_in_top_3813_DEGs': 470, 'Overlap_in_top_100_DEGs': 9, 'Overlap_in_top_50_DEGs': 6, 'Overlap_in_top_20_DEGs': 3, 'Jaccard': 0.10266491917868065}
----------------------------------------------------------------------------------------------------
Cell: k562, Pert: RPL7
DEGs Overlaps: {'Overlap_in_top_4226_DEGs': 419, 'Overlap_in_top_100_DEGs': 6, 'Overlap_in_top_50_DEGs': 3, 'Overlap_in_top_20_DEGs': 0, 'Jaccard': 0.07146512024560805}
----------------------------------------------------------------------------------------------------
Cell: k562, Pert: RNF113A
DEGs Overlaps: {'Overlap_in_top_3729_DEGs': 1203, 'Overlap_in_top_100_DEGs': 22, 'Overlap_in